# HW3 Image Classification

## Get Data

### Download Data

In [ ]:
# !curl -L "https://www.dropbox.com/s/6l2vcvxl54b0b6w/food11.zip" -o food11.zip


### Manually Unzip

## Preparation

### Importing

In [ ]:
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

from tqdm.auto import tqdm, trange
import random

from torch.utils.tensorboard import SummaryWriter


### Transforms

In [ ]:
normalize_tfm = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_tfm_without_norm = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
])

train_tfm = transforms.Compose([train_tfm_without_norm, normalize_tfm])

test_tfm_without_norm = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

test_tfm = transforms.Compose([test_tfm_without_norm, normalize_tfm])

tta_tfm = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomRotation(30)])


### Define Dataset

In [ ]:
class FoodDataset(Dataset):

    def __init__(self, path, tfm=test_tfm, files=None):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([
            os.path.join(path, x)
            for x in os.listdir(path)
            if x.endswith(".jpg")
        ])
        if files:
            self.files = files
        print(f"One {path} sample", self.files[0])
        self.transform = tfm

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        file_name = self.files[index]
        image = Image.open(file_name)
        image = self.transform(image)

        try:
            # name of an image:  .../[label]_[id].jpg
            base_name = os.path.basename(file_name)
            label = int(base_name.split("_")[0])
        except:
            label = -1
        return image, label


### Define Model

In [ ]:
class Classifier(nn.Module):

    def __init__(self, dropout_rate=0.0):
        super(Classifier, self).__init__()

        # input dim: [3, 128, 128]
        model = models.efficientnet_b0(
            weights=models.EfficientNet_B0_Weights.DEFAULT)
        num_features = model.classifier[-1].out_features
        model.classifier.append(nn.Linear(num_features, 11))
        self.cnn = model

    def forward(self, x):
        out = self.cnn(x)
        return out


### Hyperparameters

In [ ]:
# device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

seed = 3407

num_epoch = 500
early_stop = 20

batch_size = 32
learning_rate = 1e-4
weight_decay = 1e-5
dropout_rate = 0.0

tta_iterations = 10
origin_weight = 5

dataset_dir = './food11'
model_path = './model.ckpt'


### Fixing seed

In [ ]:
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


same_seeds(seed)


### Model

In [ ]:
model = Classifier().to(device)


### Optimizer


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),
                              lr=learning_rate,
                              weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)


### Dataset

In [ ]:
train_set = FoodDataset(os.path.join(dataset_dir, "training"), tfm=train_tfm)
train_loader = DataLoader(train_set,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0,
                          pin_memory=True)
valid_set = FoodDataset(os.path.join(dataset_dir, "validation"), tfm=test_tfm)
valid_loader = DataLoader(valid_set,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0,
                          pin_memory=True)


In [ ]:
def run_train_epoch(model, train_loader, criterion, optimizer, device,
                    current_epoch_num, global_step_ref):
    model.train()
    epoch_train_loss = 0.0
    epoch_train_corrects = 0
    num_train_samples = 0

    batch_pbar = tqdm(train_loader,
                      leave=False,
                      desc=f"Epoch {current_epoch_num} Training")
    for features, labels in batch_pbar:
        features = features.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
        optimizer.step()

        global_step_ref[0] += 1

        preds = outputs.argmax(dim=-1)
        epoch_train_loss += loss.item() * features.size(0)
        epoch_train_corrects += (preds.detach() == labels.detach()).sum().item()
        num_train_samples += features.size(0)

    avg_epoch_train_loss = epoch_train_loss / num_train_samples if num_train_samples > 0 else 0
    avg_epoch_train_acc = epoch_train_corrects / num_train_samples if num_train_samples > 0 else 0

    return avg_epoch_train_loss, avg_epoch_train_acc


def run_validation_epoch(model, valid_loader, criterion, device,
                         current_epoch_num):
    model.eval()
    epoch_valid_loss = 0.0
    epoch_valid_corrects = 0
    num_valid_samples = 0

    batch_pbar = tqdm(valid_loader,
                      leave=False,
                      desc=f"Epoch {current_epoch_num} Validation")
    with torch.no_grad():
        for features, labels in batch_pbar:
            features = features.to(device)
            labels = labels.to(device)

            outputs = model(features)
            loss = criterion(outputs, labels)

            preds = outputs.argmax(dim=-1)
            epoch_valid_loss += loss.item() * features.size(0)
            epoch_valid_corrects += (preds.cpu() == labels.cpu()).sum().item()
            num_valid_samples += features.size(0)

    avg_epoch_valid_loss = epoch_valid_loss / num_valid_samples if num_valid_samples > 0 else 0
    avg_epoch_valid_acc = epoch_valid_corrects / num_valid_samples if num_valid_samples > 0 else 0

    return avg_epoch_valid_loss, avg_epoch_valid_acc


def update_epoch_summary(pbar_epoch, writer_tb, current_epoch, train_loss,
                         train_acc, valid_loss, valid_acc, current_lr,
                         best_acc_so_far, not_improving_count):
    writer_tb.add_scalar('Loss/train_epoch', train_loss, current_epoch)
    writer_tb.add_scalar('Accuracy/train_epoch', train_acc, current_epoch)
    writer_tb.add_scalar('Loss/valid_epoch', valid_loss, current_epoch)
    writer_tb.add_scalar('Accuracy/valid_epoch', valid_acc, current_epoch)
    writer_tb.add_scalar('LearningRate/epoch', current_lr, current_epoch)

    info_str = (
        f"BestAcc: {best_acc_so_far:.2%}, TrainLoss: {train_loss:.4f}, TrainAcc: {train_acc:.2%}, "
        f"ValidLoss: {valid_loss:.4f}, ValidAcc: {valid_acc:.2%}, LR: {current_lr:.2e}"
    )
    if not_improving_count > 0:
        info_str += f", NoImprove: {not_improving_count}ep"
    pbar_epoch.set_postfix_str(info_str)


def train_model(model,
                train_loader,
                valid_loader,
                criterion,
                optimizer,
                scheduler,
                num_epoch,
                device,
                model_path,
                early_stop,
                tensorboard_log_dir='./runs'):
    best_val_acc = 0.0
    not_improving_epochs = 0
    global_step_counter = [0]

    epoch_pbar = trange(num_epoch, desc="Total Epochs")
    tb_writer = SummaryWriter(log_dir=tensorboard_log_dir)

    for epoch in epoch_pbar:
        avg_train_loss, avg_train_acc = run_train_epoch(model, train_loader,
                                                        criterion, optimizer,
                                                        device, epoch + 1,
                                                        global_step_counter)

        avg_valid_loss, avg_valid_acc = run_validation_epoch(
            model, valid_loader, criterion, device, epoch + 1)

        if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            scheduler.step(avg_valid_loss)
        else:
            scheduler.step()

        current_learning_rate = optimizer.param_groups[0]['lr']

        if avg_valid_acc > best_val_acc:
            not_improving_epochs = 0
            best_val_acc = avg_valid_acc
            torch.save(model.state_dict(), model_path)
        else:
            not_improving_epochs += 1

        update_epoch_summary(epoch_pbar, tb_writer, epoch + 1, avg_train_loss,
                             avg_train_acc, avg_valid_loss, avg_valid_acc,
                             current_learning_rate, best_val_acc,
                             not_improving_epochs)

        if not_improving_epochs >= early_stop:
            print(f"\nEarly stopping triggered after {epoch + 1} epochs.")
            break

    tb_writer.close()
    print(f"\nTraining finished. Best validation accuracy: {best_val_acc:.4%}")
    if best_val_acc > 0 or not_improving_epochs < num_epoch:
        print(f"Best model saved to {model_path}")
    else:
        print(
            f"No model was saved as validation accuracy did not improve over initial. Check {model_path} for pre-existing files."
        )
    return best_val_acc


## Training

In [ ]:
train_model(model, train_loader, valid_loader, criterion, optimizer, scheduler,
            num_epoch, device, model_path, early_stop)


# Test & Predict

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/  --port 6006


In [ ]:
test_set = FoodDataset(os.path.join(dataset_dir, "test"), tfm=test_tfm)
test_loader = DataLoader(test_set,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=0,
                         pin_memory=True)


In [ ]:
best_model = Classifier().to(device)
best_model.load_state_dict(torch.load("model.ckpt"))
best_model.eval()
prediction = []
with torch.no_grad():
    for image, _ in tqdm(test_loader, desc="Predicting"):
        origin_image = image.to(device)
        origin_pred = best_model(origin_image)
        origin_label = torch.argmax(origin_pred, dim=1)
        voting_predictions = [origin_label] * origin_weight

        for _ in range(tta_iterations):
            transformed_image = tta_tfm(image)
            transformed_image = transformed_image.to(device)

            tta_pred = best_model(transformed_image)
            tta_label = torch.argmax(tta_pred, dim=1)
            voting_predictions.append(tta_label)

        stacked_predictions = torch.stack(voting_predictions, dim=1)
        vote_res = torch.mode(stacked_predictions, dim=1).values
        vote_label = vote_res.cpu().data.numpy()

        prediction += vote_label.squeeze().tolist()


In [ ]:
#create test csv
def pad4(i):
    return "0" * (4 - len(str(i))) + str(i)


df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1, len(test_set) + 1)]
df["Category"] = prediction
df.to_csv("submission.csv", index=False)
